In [1]:
import torch
from torchvision.transforms import ToTensor
from torchvision import datasets
from pysr import PySRRegressor
from model import CNN
import pandas as pd
import numpy as np
from scipy.fftpack import dct, idct
import matplotlib.pyplot as plt

/home/croniev/Code/cysec/lib/python3.9/site-packages/juliacall/__init__.py:61: UserWarning: torch was imported before juliacall. This may cause a segfault. To avoid this, import juliacall before importing torch. For updates, see https://github.com/pytorch/pytorch/issues/78829.
  warnings.warn(


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


### Load Model and get Kernels

In [2]:
cnn = CNN()
cnn.load_state_dict(torch.load('cnn.pt'))
print(cnn)

for name, param in cnn.named_parameters():
    if name == 'conv1.weight':
        print(f"amount of kernels of Conv1: {param.shape}")
        kernels1 = param
    if name == 'conv2.weight':
        print(f"amount of kernels of Conv2: {param.shape}")
print(f"kernels of first layer:\n{kernels1}")

/tmp/ipykernel_96769/3040839533.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn.load_state_dict(torch.load('cnn.pt'))


CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (out): Linear(in_features=1568, out_features=10, bias=True)
)
amount of kernels of Conv1: torch.Size([16, 1, 5, 5])
amount of kernels of Conv2: torch.Size([32, 16, 5, 5])
kernels of first layer:
Parameter containing:
tensor([[[[-0.2335, -0.2440, -0.9097, -1.0703, -0.8168],
          [-0.0917,  0.0114, -0.2991, -0.3230,  0.6103],
          [ 0.0865,  0.0287,  0.0781,  0.3111,  0.9103],
          [ 0.0553, -0.7562, -0.8900, -0.9722, -0.9644],
          [ 0.0107, -0.6425, -0.8105, -0.5706, -1.1133]]],


        [[[-0.0280, -0.6446, -1.0783, -0.0510, -0.1909],
          [-1.2921, -1.5182, -0.0516,  0.5699, -0.5167],
          [-

### Load Dataset and get results

In [3]:
test_data = datasets.MNIST(root='data', train=False, transform=ToTensor(),)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=20, shuffle=True, num_workers=1)
samples, labels = next(iter(test_loader))

cnn.eval()
with torch.no_grad():
    results = cnn(samples)

### Prepare Data for PySR and extract features
Extract the 5x5 submatrices (incl. padding) from images to use them as input

In [4]:
# Take every image and split it into 5x5 submatrices => np.array.shape = (7840, 5, 5)
# 7840 <- 28 * 28 patches per image * 10 images (batch_size)
kernel_size = 5
X = None
for x in samples:
    x = torch.nn.functional.pad(input=x[0], pad=(2, 2, 2, 2), mode="constant", value=0)
    for i, j in np.ndindex((x.size()[0] - kernel_size + 1, x.size()[1] - kernel_size + 1)):
        slice = x[i:i + kernel_size, j:j + kernel_size]
        if X is None:
            # X = np.array([slice.numpy().flatten()])
            X = np.array([slice.numpy()])
        else:
            # X = np.concatenate((X, [slice.numpy().flatten()]))
            X = np.concatenate((X, [slice.numpy()]))

print(X.shape)

# Get the result for every 5x5 submatrix for each kernel => np.array.shape = (16, 7840)
# 16 <- amount of kernels in the first layer
y = results['relu1'].numpy().transpose(1, 0, 2, 3).reshape(16, X.shape[0])
print(y.shape)

(15680, 5, 5)
(16, 15680)


#### Discrete Cosine Transform (DCT, Fourier Transform)
Get DCT features of image, only keep low frequencies.

In [5]:
def dct2(m):
    return dct(dct(m.T, norm='ortho').T, norm='ortho')

Xdct = np.array(list(map(dct2, X)))
dct_filter_param = 3
Xdct_filtered = np.array(list(map(lambda m: m[:dct_filter_param, :dct_filter_param], Xdct)))
print(Xdct_filtered.shape)

(15680, 3, 3)


### Symbolic Regression

In [6]:
def newPySRRegressor():
    return PySRRegressor(
        niterations=40,
        binary_operators=["+", "*", "-", "/"],
        unary_operators=[
            "cos",
            "exp",
            "sin",
            "square",
            "cube",
            "inv(x) = 1/x",  # Julia syntax
        ],
        extra_sympy_mappings={"inv": lambda x: 1 / x},  # Sympy syntax
        elementwise_loss="loss(prediction, target) = (prediction - target)^2",  # Julia syntax
        warm_start=False,
        verbosity=0,
        temp_equation_file=True,
    )

#### Over the first kernel

In [7]:
regr_first_kernel = newPySRRegressor()
regr_first_kernel.fit(Xdct_filtered.reshape(X.shape[0], dct_filter_param**2), y[0])  # Input data coded for position and summed

/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                 sin(0.00043246243)   
	1        0.010580                                  -0.004426408 * x6   
	2        0.016973                            cube(x6) * -0.004426408   
	3        0.002326                      square(cube(0.30356145 * x6))   
	4        0.004006                       -0.004841388 * cube(x6 + x8)   
	5        0.000390                  -0.004841388 * cube(x6 + sin(x8))   
	6  >>>>  0.026873     square(square(cube(x6)) * (x3 * -0.007614721))   
	7        0.013111  square(square(cube(x6)) * ((x3 * -0.007614721)...   
	
	       loss  complexity  
	0  0.000171           2  
	1  0.000169           3  
	2  0.000166           4  
	3  0.000166           5  
	4  0.000165           6  
	5  0.000165           7  
	6  0.000161           8  
	7  0.000156          10  
]

In [ ]:
print(regr_first_kernel.equations_['lambda_format'])
f = regr_first_kernel.equations_['lambda_format'][4]
print(f(np.array(range(9))))

#### Over all 16 Kernels

In [ ]:
regr_all_kernels = pd.DataFrame()
regr_all_kernels.index.names = ['complexity']
for i in range(16):
    regr = newPySRRegressor()
    regr.fit(Xdct_filtered.reshape((X.shape[0], dct_filter_param**2)), y[i])  # Input data as is
    # print(regr.equations_)
    regr_all_kernels.insert(loc=i, column=f'Kernel {i}', value=regr.equations_['equation'])
    print(f"Done with Kernel {i} | {(i+1)/16 * 100}%")

print(regr_all_kernels)

In [ ]:
regr_all_kernels.to_csv('results_dct_filter3_allkernels.csv')

#### Over the first kernel multiple times (Stability check)

In [ ]:
regr_stability = pd.DataFrame()
regr_stability.index.names = ['complexity']
for i in range(10):
    regr = newPySRRegressor()
    regr.fit(Xdct_filtered.reshape((X.shape[0], dct_filter_param**2)), y[0])
    # print(regr.equations_)
    regr_stability.insert(loc=i, column=f'Iteration {i}', value=regr.equations_['equation'])
    # print(regr_stability)
    print(f"Done with Iteration {i} | {(i+1)/10 * 100}%")

In [ ]:
regr_stability.to_csv('results_dct_filter3_stability.csv')